In [30]:
# Implementation of a simple Stack and an ArrayList with undo functionality.

class Stack:
    """A simple stack implementation using a linked list representation."""
    
    def __init__(self):
        # Initialize an empty stack with size 0 and no elements.
        self.inList = None
        self.size = 0
        
    def push(self, v):
        """Push a value onto the stack."""
        self.size += 1
        # Store the new value and link it to the current stack.
        self.inList = (v, self.inList)
        
    def pop(self):
        """Pop the top value from the stack."""
        # Assert ensures the stack is not empty before popping.
        if self.size == 0: assert(0)
        self.size -= 1
        # Retrieve the top value and update the stack pointer.
        (v, ls) = self.inList
        self.inList = ls
        return v
    
    def __str__(self):
        """Return a string representation of the stack."""
        s = "["
        ls = self.inList
        for _ in range(self.size):
            (v, ls) = ls
            s += str(v)
            if ls != None: s += ", "
        return s + "]"
        
class ArrayList:
    """A dynamic array-like data structure."""
    
    def __init__(self):
        # Initialize with a fixed capacity array and count of elements.
        self.inArray = [0 for _ in range(10)]
        self.count = 0
        
    def get(self, i):
        """Get the element at index i."""
        return self.inArray[i]

    def set(self, i, e):
        """Set the element at index i to e."""
        self.inArray[i] = e

    def length(self):
        """Return the current number of elements."""
        return self.count

    def append(self, e):
        """Add an element to the end of the array."""
        self.inArray[self.count] = e
        self.count += 1
        # Resize the array if capacity is reached.
        if len(self.inArray) == self.count:
            self._resizeUp()

    def insert(self, i, e):
        """Insert an element at index i, shifting subsequent elements."""
        for j in range(self.count, i, -1):
            self.inArray[j] = self.inArray[j-1]
        self.inArray[i] = e
        self.count += 1
        if len(self.inArray) == self.count:
            self._resizeUp()
    
    def remove(self, i):
        """Remove the element at index i and shift subsequent elements."""
        self.count -= 1
        val = self.inArray[i]
        for j in range(i, self.count):
            self.inArray[j] = self.inArray[j+1]
        return val

    def _resizeUp(self):
        """Double the array size when capacity is exceeded."""
        newArray = [0 for _ in range(2 * len(self.inArray))]
        for j in range(len(self.inArray)):
            newArray[j] = self.inArray[j]
        self.inArray = newArray
        
    def toArray(self):
        """Return a list representation of the current elements."""
        return self.inArray[:self.count]

    def __str__(self):
        """Return a string representation of the array."""
        if self.count == 0: return "[]"
        s = "["
        for i in range(self.count-1): s += str(self.inArray[i]) + ", "
        return s + str(self.inArray[self.count-1]) + "]"

class ArrayListWithUndo(ArrayList):
    """Extension of ArrayList to support undo operations."""
    
    def __init__(self):
        # Initialize ArrayList and a stack for undo operations.
        super().__init__()
        self.undos = Stack()

    def set(self, i, v):
        """Set element at index i and store undo information."""
        self.undos.push(("set", i, self.inArray[i]))
        self.inArray[i] = v

    def append(self, v):
        """Append an element and store undo information."""
        self.undos.push(("rem", self.count, None))
        super().append(v)

    def remove(self, i):
        """Remove element at index i and store undo information."""
        if i < self.count:  # Check if i is within bounds
            self.undos.push(("ins", i, self.inArray[i]))
            super().remove(i)

    def insert(self, i, v):
        """Insert element at index i and store undo information."""
        if i <= self.count:  # Ensure valid index
            self.undos.push(("rem", i, None))
            super().insert(i, v)

    def undo(self):
        """Undo the last operation."""
        if self.undos.size == 0:
            return

        op, i, v = self.undos.pop()

        if op == "set":
            self.inArray[i] = v  # Restore previous value
        elif op == "rem":
            if self.count == 0:
                self.count = 0
            else:
                self.count -= 1  # Undo an append operation
        elif op == "ins":
            self.insert(i, v)  # Reinsert a removed element

    def __str__(self):
        """Return string representation of the list and undo stack."""
        return str(self.toArray()) + "\n-> " + str(self.undos)
    def __getitem__(self, index):
        return self.inArray[index]

    def __setitem__(self, index, value):
        self.inArray[index] = value

    def __len__(self):
        return len(self.inArray)


# Minimal tests for ArrayListWithUndo

def tprint(ls, i):
    """Print test case results."""
    print("\n=== Test", i, "===\n", ls, "\nsize", ls.length())

ls = ArrayListWithUndo()
A = [2, 3, 4, 5, 5, 1, 4]
for x in A: 
    ls.append(x)
ls.set(4, 2)
ls.insert(3, 10)
ls.remove(0)
tprint(ls, 0)
for i in range(len(A) + 4):
        ls.undo()
        tprint(ls, i + 1)



=== Test 0 ===
 [3, 4, 10, 5, 2, 1, 4]
-> [('ins', 0, 2), ('rem', 3, None), ('set', 4, 5), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)] 
size 7

=== Test 1 ===
 [2, 3, 4, 10, 5, 2, 1, 4]
-> [('rem', 0, None), ('rem', 3, None), ('set', 4, 5), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)] 
size 8

=== Test 2 ===
 [2, 3, 4, 10, 5, 2, 1]
-> [('rem', 3, None), ('set', 4, 5), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)] 
size 7

=== Test 3 ===
 [2, 3, 4, 10, 5, 2]
-> [('set', 4, 5), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)] 
size 6

=== Test 4 ===
 [2, 3, 4, 10, 5, 2]
-> [('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1,

In [31]:
class Node:
 def __init__(self, d, n):
    self.data = d
    self.next = n

class linkedList:
    def __init__(self):
        self.head = None
        self.length = 0

    def search(self, d):
        i = 0
        ptr = self.head
        while ptr is not None:
            if ptr.data == d:
                return i
            ptr = ptr.next
            i += 1
        return -1

    def append(self, d):
        if self.head is None:
            self.head = Node(d, None)
        else:
            ptr = self.head
            while ptr.next is not None:
                ptr = ptr.next
            ptr.next = Node(d, None)
        self.length += 1

    def insert(self, i, d):
        if self.head is None or i == 0:
            self.head = Node(d, self.head)
        else:
            ptr = self.head
            while i > 1 and ptr.next is not None:
                ptr = ptr.next
                i -= 1
            ptr.next = Node(d, ptr.next)
        self.length += 1

    def remove(self, i):  # removes/returns i-th element
        if self.head is None:
            return None
        if i == 0:
            val = self.head.data
            self.head = self.head.next
            self.length -= 1
            return val
        else:
            ptr = self.head
            while i > 1 and ptr.next is not None:
                ptr = ptr.next
                i -= 1
            if i == 1 and ptr.next is not None:
                val = ptr.next.data
                ptr.next = ptr.next.next
                self.length -= 1
                return val
            return None
    def __str__(self):
        values = []
        current = self.head
        while current:
            values.append(str(current.value))  # Convert the node value to string
            current = current.next
        return f"[{', '.join(values)}]"  # Return as a list-like string
        
    def get_elements(self):
        # Return a list of all values in the LinkedList
        elements = []
        current = self.head
        while current:
            elements.append(current.value)
            current = current.next
        return elements

In [46]:
class NetworkWithUndo:
    def __init__(self, N):
        # already implemented
        self.inArray = ArrayListWithUndo()
        for _ in range(N): 
            self.inArray.append(-1)
        self.undos = Stack()
        self.undos.push(N)

    def getSize(self):
        # already implemented
        return self.inArray.length()

    def add(self):
        self.inArray.append(-1)
        self.undos.push(1) 

    def root(self, i):
        if i < 0 or i >= self.inArray.length():
            raise IndexError(f"Node {i} is out of bounds.")


        path = []
        while self.inArray.get(i) >= 0:
            path.append(i)
            i = self.inArray.get(i)
        
        # Path compression
        for node in path:
            self.inArray.set(node,i)
            
            
        self.undos.push(1)
            #count += 1

        return i

    def merge(self, i, j):
        if self.inArray.get(i) > 0 and self.inArray.get(j) > 0:
            assert(0)

        if self.inArray.get(i) < self.inArray.get(j):
            self.inArray.set(i, self.inArray.get(i) + self.inArray.get(j))
            self.inArray.set(j, i)
        else:
            self.inArray.set(j, self.inArray.get(i) + self.inArray.get(j))
            self.inArray.set(i, j)
        self.undos.push(2)
        pass

    def undo(self):
        if self.undos.size == 0:
            return
        
        last_op = self.undos.pop()
        
        for x in range(last_op):
            self.inArray.undo()
            pass

    def toArray(self):
        # already implemented
        return self.inArray.toArray()
        
    def __str__(self):
        return str(self.toArray()) + "\n-> " + str(self.undos)

In [47]:
# Create a network with 5 initial nodes
net = NetworkWithUndo(5)
print("=== Test 0 ===")
print(net)  # Expected: [-1, -1, -1, -1, -1]
print("Size:", net.getSize())  # Expected: 5

# Test 1: Get the root of node 0, should return -1 because node 0 is its own root
print("\n=== Test 1 ===")
print("Root of 0:", net.root(0))  # Expected: -1

# Test 2: Get the root of node 4, should also return -1 as node 4 is its own root
print("\n=== Test 2 ===")
print("Root of 4:", net.root(4))  # Expected: -1

# Test 3: Add a new node to the network
net.add()
print("\n=== Test 3 ===")
print(net)  # Expected:  [-1, -1, -1, -1, -1, -1] (new node added)
print("Size:", net.getSize())  # Expected: 6

# Test 4: After adding, get the root of the newly added node (node 5), should return -1 as it is its own root
print("\n=== Test 4 ===")
print("Root of 5:", net.root(5))  # Expected: -1

# Test 5: Add another new node to the network
net.add()
print("\n=== Test 5 ===")
print(net)  # Expected: [-1, -1, -1, -1, -1, -1, -1] (new node added)
print("Size:", net.getSize())  # Expected: 7

# Test 6: Get the root of node 6, should return -1 as it is its own root
print("\n=== Test 6 ===")
print("Root of 6:", net.root(6))  # Expected: -1

try:
    print("\n=== Test 3 ===")
    print("Root of 10:", net.root(10))  # Expected: Error (out of bounds)
except Exception as e:
    print(f"Error: {e}")


=== Test 0 ===
[-1, -1, -1, -1, -1]
-> [5]
Size: 5

=== Test 1 ===
Root of 0: 0

=== Test 2 ===
Root of 4: 4

=== Test 3 ===
[-1, -1, -1, -1, -1, -1]
-> [1, 1, 1, 5]
Size: 6

=== Test 4 ===
Root of 5: 5

=== Test 5 ===
[-1, -1, -1, -1, -1, -1, -1]
-> [1, 1, 1, 1, 1, 5]
Size: 7

=== Test 6 ===
Root of 6: 6

=== Test 3 ===
Error: Node 10 is out of bounds.


In [48]:
# minimal tests NetworkWithUndo

def tprint(n,i,s=""):
    print("\n=== Test",i,"===")
    if s!="": print(s)
    print(n,"\nsize",n.getSize())

net = NetworkWithUndo(9)
tprint(net,0)
net.merge(4,0); net.merge(0,3); net.merge(2,1); net.merge(6,8); net.merge(8,7)
tprint(net,1)
net.merge(0,8)
tprint(net,2)
x = net.root(4)
tprint(net,3,"root of 4: "+str(x))
net.undo()
tprint(net,4)
net.undo()
tprint(net,5)
net.merge(0,1)
tprint(net,6)
for i in range(8): 
    net.undo(); tprint(net,i+6)


=== Test 0 ===
[-1, -1, -1, -1, -1, -1, -1, -1, -1]
-> [9] 
size 9

=== Test 1 ===
[-3, -2, 1, 0, 0, -1, 8, 8, -3]
-> [2, 2, 2, 2, 2, 9] 
size 9

=== Test 2 ===
[8, -2, 1, 0, 0, -1, 8, 8, -6]
-> [2, 2, 2, 2, 2, 2, 9] 
size 9

=== Test 3 ===
root of 4: 8
[8, -2, 1, 0, 8, -1, 8, 8, -6]
-> [1, 2, 2, 2, 2, 2, 2, 9] 
size 9

=== Test 4 ===
[8, -2, 1, 0, 8, -1, 8, 8, -6]
-> [2, 2, 2, 2, 2, 2, 9] 
size 9

=== Test 5 ===
[-3, -2, 1, 0, 0, -1, 8, 8, -6]
-> [2, 2, 2, 2, 2, 9] 
size 9

=== Test 6 ===
[-5, 0, 1, 0, 0, -1, 8, 8, -6]
-> [2, 2, 2, 2, 2, 2, 9] 
size 9

=== Test 6 ===
[-3, -2, 1, 0, 0, -1, 8, 8, -6]
-> [2, 2, 2, 2, 2, 9] 
size 9

=== Test 7 ===
[-3, -2, 1, 0, 0, -1, 8, -1, -3]
-> [2, 2, 2, 2, 9] 
size 9

=== Test 8 ===
[-3, -2, 1, 0, 0, -1, -1, -1, -2]
-> [2, 2, 2, 9] 
size 9

=== Test 9 ===
[-3, -2, -1, 0, 0, -1, -1, -1, -1]
-> [2, 2, 9] 
size 9

=== Test 10 ===
[-3, -1, -1, -1, 0, -1, -1, -1, -1]
-> [2, 9] 
size 9

=== Test 11 ===
[-2, -1, -1, -1, -1, -1, -1, -1, -1]
-> [9] 
size 9
